# Setup

In [1]:
# TODO: add annotations describing usage of different modules

from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer
from sklearn.svm import SVR
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import altair as alt
import random
import warnings

warnings.filterwarnings('ignore')

# set seaborn whitegrid theme
sns.set(style="whitegrid")

from sklearn.inspection import permutation_importance
from random import sample


/Users/orencarmeli/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')
print(cdc_survey.shape)

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
print(cdc_survey_pmom.shape)

(35706, 863)
(7741, 863)


In [3]:
# Depression screener
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]

# Alcohol & smoking
alcohol_n_smoking_cols = [
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo', 
    'drank_alc',
    'alc_drinking_freq',
    'alc_per_day',
    'times_with_4or5_alc',
    'times_with_8plus_alc',
    'times_with_12plus_alc',
    '4plus_alc_daily',
    'days_4plus_drinks_occasion',
    'smoked_100_cigs',
    'currently_smoke'
]

#Medical Conditions 
med_condition_cols = [
    'chest_discomfort',
    'asthma',
    'anemia_treatment',
    'blood_transfusion',
    'arthritis',
    'heart_failure',
    'coronary_heart_disease',
    'angina_pectoris',
    'heart_attack',
    'stroke',
    'thyroid_issues',
    'respiratory_issues',
    'abdominal_pain',
    'gallstones',
    'gallbladder_surgery',
    'cancer',
    'dr_recommend_lose_weight',
    'dr_recommend_exercise',
    'dr_recommend_reduce_salt',
    'dr_recommend_reduce_fat',
    'currently_losing_weight',
    'currently_increase_exercise',
    'currently_reducing_salt',
    'currently_reducing_fat',
    'metal_objects',
    'has_diabetes',    
    #Blood Pressure & Cholesterol 
    'high_bp',
    'age_hypertension',
    'hypertension_prescription',
    'high_bp_prescription',
    'high_cholesterol',
    'cholesterol_prescription',
    # general health
    'has_overweight_diagnosis',   
    'height_in',
    'weight_lbs', 
    'general_health_condition',      
]

# diet, nutrition, & exercise 
lifestyle_cols = [
    # exercise/weight loss
    'vigorous_work',
    'walk_or_bicycle',
    'vigorous_recreation',
    'moderate_recreation',    
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo', 
    'attempt_weight_loss_1yr',           
    # diet
    'how_healthy_is_your_diet',    
    'milk_consumption_freq',
    'govmnt_meal_delivery',
    'nonhomemade_meals',
    'fastfood_meals',
    'readytoeat_meals',
    'frozen_pizza'   
]

# Reproductive Health 
reproductive_health_cols = [
    'regular_periods',
    'age_last_period',
    'try_pregnancy_1yr',
    'see_dr_fertility',
    'pelvic_infection',
    'pregnant_now',
    'pregnancy_count',
    'diabetes_pregnancy',
    'delivery_count',
    'live_birth_count',
    'age_at_first_birth',
    'age_at_last_birth',
    'months_since_birth',
    'horomones_not_bc'
]

# hospital usage & access to care
hospital_utilization_n_access_cols = [
    'general_health',
    'regular_healthcare_place',
    'time_since_last_healthcare',
    'overnight_in_hospital',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',   
    'have_health_insurance',
    'have_private_insurance',
    'has_health_insurance_gap',       
    'plan_cover_prescriptions'    
]

# socioeconomic status
socioeconomic_cols = [
    #Food Security
    'emergency_food_received',
    'food_stamps_used',      
    #Income
    'family_poverty_level',
    'family_poverty_level_category',
    #Occupation
    'hours_worked',
    'over_35_hrs_worked',
    'work_schedule'    
]

demographic_cols = [
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born'
]

# combine all candidate features
candidate_features = dep_screener_cols + \
    alcohol_n_smoking_cols + \
    med_condition_cols + \
    hospital_utilization_n_access_cols + \
    lifestyle_cols + \
    socioeconomic_cols + \
    reproductive_health_cols + \
    demographic_cols


len(candidate_features)

115

In [4]:
## ###############################################################################
## function to create test & train dataset

def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False,
                   return_indices=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    #cols_to_use.insert(0, 'SEQN')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    indices = np.arange(y.shape[0])
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split(
        x, 
        y, 
        indices,
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    if return_indices:
        return x_train, x_test, y_train, y_test, idx_train, idx_test
    else:
        return x_train, x_test, y_train, y_test

## ###############################################################################
## Function to get performance across different models

def get_performance_df(label_actual, label_pred, model_name):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    # create classification report
    result_table = classification_report(label_actual, label_pred, output_dict=True)
    result_table = pd.DataFrame.from_dict(result_table)

    # store for later
    accuracies = result_table['accuracy'][0]
    
    column_key = {
        '0':'Depressed (No)',
         '1':'Depressed (Yes)',
         'accuracy':'accuracy',
         'macro avg':'Macro Avg',
         'weighted avg':'Weighted Avg'
    }

    # rename grouping
    result_table.columns = [column_key.get(key) for key in result_table.columns]

    # create dataframe with 1 row per grouping
    result_table.drop(labels = 'accuracy', axis = 1, inplace=True)
    result_table = result_table.transpose()
    result_table['accuracy'] = [accuracies for i in range(result_table.shape[0])]
    result_table = result_table.reset_index()
    result_table.rename(columns = {'index':'grouping'},inplace=True)
    result_table['model'] = model_name
    result_table = result_table[['model','grouping','precision','recall','f1-score','support','accuracy']]
    return result_table

In [5]:
cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)

# Opt 1: Best Single Model (Include All Observations)


In [6]:
model_features_opt1 = dep_screener_cols + ['times_with_12plus_alc',
 'seen_mental_health_professional',
 'count_days_seen_doctor_12mo',
 'count_lost_10plus_pounds',
 'arthritis',
 'horomones_not_bc',
 'is_usa_born',
 'times_with_8plus_alc',
 'time_since_last_healthcare',
 'duration_last_healthcare_visit',
 'work_schedule',
 'age_in_years',
 'regular_periods',
 'count_minutes_moderate_sedentary_activity',
 'emergency_food_received',
 'high_bp',
 'dr_recommend_exercise',
 'metal_objects',
 'drank_alc',
 'cholesterol_prescription',
 'smoked_100_cigs',
 'vigorous_recreation',
 'dr_recommend_lose_weight',
 'cancer',
 'chest_discomfort',
 'have_health_insurance',
 'weight_lbs',
 'high_cholesterol']

x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = model_features_opt1,
    return_indices = True
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

gnb_opt1 = GaussianNB()
gnb_opt1.fit(x_train, y_train)
gnb_opt1_pred  = gnb_opt1.predict(x_test)
gnb_opt1_score = get_performance_df(y_test, gnb_opt1_pred,'Gaussian Naive Bayes')  
gnb_opt1_score

(6192, 38)
(1549, 38)
(6192,)
(1549,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes,Depressed (No),0.951777,0.809353,0.874806,1390.0,0.792124
1,Gaussian Naive Bayes,Depressed (Yes),0.277929,0.641509,0.387833,159.0,0.792124
2,Gaussian Naive Bayes,Macro Avg,0.614853,0.725431,0.631319,1549.0,0.792124
3,Gaussian Naive Bayes,Weighted Avg,0.882608,0.792124,0.824819,1549.0,0.792124


**Performance on those with 1+ dep screener answered**

In [7]:
gnb_opt1_pred_prob = pd.DataFrame(gnb_opt1.predict_proba(x_test))
gnb_opt1_pred_prob.columns = ['depressed_no_prob','depressed_yes_prob']
gnb_opt1_pred_prob['y_pred'] = gnb_opt1_pred
gnb_opt1_pred_prob['y_actual'] = y_test

gnb_opt1_pred_prob_with_feat = pd.concat([
    gnb_opt1_pred_prob, 
    # column bind test observations with their original values
    cdc_survey_pmom.iloc[idx_test][model_features_opt1].reset_index()
], axis=1)

gnb_opt1_pred_prob_with_feat['num_dep_screener_0'] = (gnb_opt1_pred_prob_with_feat[dep_screener_cols]==0).sum(axis=1)

In [8]:
replied_to_1plus_dep_screener = gnb_opt1_pred_prob_with_feat[
    gnb_opt1_pred_prob_with_feat['num_dep_screener_0'] < 9
]

get_performance_df(
    replied_to_1plus_dep_screener['y_actual'], 
    replied_to_1plus_dep_screener['y_pred'],
    'GNB: Only those with 1 dep screener replied'
)  

,model,grouping,precision,recall,f1-score,support,accuracy
0,GNB: Only those with 1 dep screener replied,Depressed (No),0.929104,0.675712,0.782404,737.0,0.680139
1,GNB: Only those with 1 dep screener replied,Depressed (Yes),0.275758,0.705426,0.396514,129.0,0.680139
2,GNB: Only those with 1 dep screener replied,Macro Avg,0.602431,0.690569,0.589459,866.0,0.680139
3,GNB: Only those with 1 dep screener replied,Weighted Avg,0.831781,0.680139,0.724921,866.0,0.680139


# Opt 2: Best Single Model (Only Observations with 1+ dep screener resposnse)

In [9]:
cdc_survey_pmom['weight_lbs_over_height_in_ratio'] = round(cdc_survey_pmom['weight_lbs'] / cdc_survey_pmom['height_in'],1)

model_features_opt2 = dep_screener_cols + ['seen_mental_health_professional',
 'times_with_12plus_alc',
 'time_since_last_healthcare',
 'cholesterol_prescription',
 'high_cholesterol',
 'age_in_years',
 'horomones_not_bc',
 'months_since_birth',
 'arthritis',
 'high_bp',
 'regular_periods',
 'moderate_recreation',
 'thyroid_issues',
 'vigorous_recreation',
 'stroke',
 'is_usa_born',
 'asthma',
 'count_days_moderate_recreational_activity',
 'have_health_insurance',
 'num_dep_screener_0',
 'weight_lbs_over_height_in_ratio'
]

x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9],
    columns = model_features_opt2,
    return_indices = True
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

gnb_opt2 = GaussianNB()
gnb_opt2.fit(x_train, y_train)
gnb_opt2_pred  = gnb_opt2.predict(x_test)
gnb_opt2_score = get_performance_df(y_test, gnb_opt2_pred,'Gaussian Naive Bayes (Subset)')  
gnb_opt2_score

(3515, 31)
(879, 31)
(3515,)
(879,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes (Subset),Depressed (No),0.930757,0.782138,0.850000,739.0,0.767918
1,Gaussian Naive Bayes (Subset),Depressed (Yes),0.375969,0.692857,0.487437,140.0,0.767918
2,Gaussian Naive Bayes (Subset),Macro Avg,0.653363,0.737498,0.668719,879.0,0.767918
3,Gaussian Naive Bayes (Subset),Weighted Avg,0.842395,0.767918,0.792254,879.0,0.767918


In [10]:
model_features_opt2

['little_interest_in_doing_things',
 'feeling_down_depressed_hopeless',
 'trouble_falling_or_staying_asleep',
 'feeling_tired_or_having_little_energy',
 'poor_appetitie_or_overeating',
 'feeling_bad_about_yourself',
 'trouble_concentrating',
 'moving_or_speaking_to_slowly_or_fast',
 'thoughts_you_would_be_better_off_dead',
 'difficult_doing_daytoday_tasks',
 'seen_mental_health_professional',
 'times_with_12plus_alc',
 'time_since_last_healthcare',
 'cholesterol_prescription',
 'high_cholesterol',
 'age_in_years',
 'horomones_not_bc',
 'months_since_birth',
 'arthritis',
 'high_bp',
 'regular_periods',
 'moderate_recreation',
 'thyroid_issues',
 'vigorous_recreation',
 'stroke',
 'is_usa_born',
 'asthma',
 'count_days_moderate_recreational_activity',
 'have_health_insurance',
 'num_dep_screener_0',
 'weight_lbs_over_height_in_ratio']

In [11]:
len(model_features_opt2)

31

## Callout

There are 5 features in op2 not in opt1. If we use these as ensemble will need to think about how we deal with this. I.E maybe we can market it where we start with 10 dep screener cols. Based on their response we use either X or Y additional cols.

In [12]:
[feature for feature in model_features_opt2 if feature not in model_features_opt1]

['months_since_birth',
 'moderate_recreation',
 'thyroid_issues',
 'stroke',
 'asthma',
 'count_days_moderate_recreational_activity',
 'num_dep_screener_0',
 'weight_lbs_over_height_in_ratio']

# Opt 3: Ensemble Model

Build 2 models both use feature set from model opt 1
- Model 1: 
 - GNB trained on the entire dataset
- Model 2: 
 - DT trained on observations where the respondent answers 0 for 9+ of the depression screeners

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [13]:
# def get_model_data_ensemble_v1(original_df, 
#                             columns_to_use):
#     """
#     Function to build feature & indicator matrices for both train & test.
#     """
    
#     data_low_dep_screener = original_df[original_df['num_dep_screener_0'] >= 9].copy()
#     data_high_dep_screener = original_df[original_df['num_dep_screener_0'] < 9].copy()
    
#     x_train_low, x_test_low, y_train_low, y_test_low = get_model_data(
#         original_df = data_low_dep_screener,
#         columns = columns_to_use
#     )
    
#     x_train_high, x_test_high, y_train_high, y_test_high = get_model_data(
#         original_df = data_high_dep_screener,
#         columns = columns_to_use
#     )
    
#     return x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high

# x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v1(
#     original_df = cdc_survey_pmom,
#     columns_to_use = model_features_opt1
# )

# print(x_train_low.shape)
# print(x_test_low.shape)
# print(y_train_low.shape)
# print(y_test_low.shape)
# print(x_train_high.shape)
# print(x_test_high.shape)
# print(y_train_high.shape)
# print(y_test_high.shape)

# ################# Model 1: Based on all datapoints
# x_train_all = np.concatenate((x_train_low, x_train_high))
# y_train_all = np.concatenate((y_train_low, y_train_high))

# gnb = GaussianNB()
# gnb.fit(x_train_all, y_train_all)

# ################# Model 2: Based on cases where >8 dep screener are 0
# dt = DecisionTreeClassifier(random_state=42)
# dt.fit(x_train_low, y_train_low)

# ################# Predict 

# # Model 1: use for high cases
# y_pred_high = gnb.predict(x_test_high)

# # Model 2: use for low cases
# y_pred_low = dt.predict(x_test_low)

# y_pred = np.concatenate((y_pred_high, y_pred_low))
# y_actual = np.concatenate((y_test_high, y_test_low))

# # Performance All
# ensemble_performance_all = get_performance_df(
#     label_actual = y_actual, 
#     label_pred = y_pred,
#     model_name = 'All Obs'
# )

# # Performance Model 1
# ensemble_performance_high = get_performance_df(
#     label_actual = y_test_high, 
#     label_pred = y_pred_high,
#     model_name = 'Obs w/ 1+ Dep Screener'
# )

# # Performance Model 2
# ensemble_performance_low = get_performance_df(
#     label_actual = y_test_low, 
#     label_pred = y_pred_low,
#     model_name = 'Obs w/ 0 Dep Screener'
# )

# ensemble_performance_opt3 = pd.concat([
#     ensemble_performance_all,
#     ensemble_performance_high,
#     ensemble_performance_low
# ])

# cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
# ensemble_performance_opt3[cols].sort_values(by=['model','grouping']) \
#     .rename(columns = {'model':'Observations (Obs)'})

# Opt 4: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GNB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2
- Model 2: 
 - DT trained on observations with 0 dep screener response. 
 - Uses features from opt 1

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [14]:
def get_model_data_ensemble_v2(original_df, 
                            columns_to_use_low,
                            columns_to_use_high):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    data_low_dep_screener = original_df[original_df['num_dep_screener_0'] >= 9].copy()
    data_high_dep_screener = original_df[original_df['num_dep_screener_0'] < 9].copy()
    
    x_train_low, x_test_low, y_train_low, y_test_low = get_model_data(
        original_df = data_low_dep_screener,
        columns = columns_to_use_low
    )
    
    x_train_high, x_test_high, y_train_high, y_test_high = get_model_data(
        original_df = data_high_dep_screener,
        columns = columns_to_use_high
    )
    
    return x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high

In [15]:
# x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
#     original_df = cdc_survey_pmom,
#     columns_to_use_low = model_features_opt1,
#     columns_to_use_high = model_features_opt2
# )

# print(x_train_low.shape)
# print(x_test_low.shape)
# print(y_train_low.shape)
# print(y_test_low.shape)
# print(x_train_high.shape)
# print(x_test_high.shape)
# print(y_train_high.shape)
# print(y_test_high.shape)
# print(len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high))

# ## #####################################################
# ## Model 1: Based on observations with 1+ dep screener non-zero response

# gnb = GaussianNB()
# gnb.fit(x_train_high, y_train_high)
# y_pred_high = gnb.predict(x_test_high)

# ## #####################################################
# ## Model 2: Based on observations with 0 dep screener non-zero response

# dt = DecisionTreeClassifier(random_state=42)
# dt.fit(x_train_low, y_train_low)
# y_pred_low = dt.predict(x_test_low)

# ## #####################################################
# ## Predict: 

# y_pred = np.concatenate((y_pred_high, y_pred_low))
# y_actual = np.concatenate((y_test_high, y_test_low))

# # Performance All
# ensemble_performance_all = get_performance_df(
#     label_actual = y_actual, 
#     label_pred = y_pred,
#     model_name = 'All Obs'
# )

# # Performance Model 1
# ensemble_performance_high = get_performance_df(
#     label_actual = y_test_high, 
#     label_pred = y_pred_high,
#     model_name = 'Obs w/ 1+ Dep Screener'
# )

# # Performance Model 2
# ensemble_performance_low = get_performance_df(
#     label_actual = y_test_low, 
#     label_pred = y_pred_low,
#     model_name = 'Obs w/ 0 Dep Screener'
# )

# ## #####################################################
# ## Performance
# ensemble_performance_opt4 = pd.concat([
#     ensemble_performance_all,
#     ensemble_performance_high,
#     ensemble_performance_low
# ])

# cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
# ensemble_performance_opt4[cols].sort_values(by=['model','grouping']) \
#     .rename(columns = {'model':'Observations (Obs)'})

# Opt 5: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GNB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2
- Model 2: 
 - GB trained on observations with 0 dep screener response. 
 - Uses separate features. 
 - Random under sampler

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [16]:
model_features_low_opt5 = [
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'work_schedule',
    'age_in_years'
]

x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_low_opt5,
    columns_to_use_high = model_features_opt2
)

print(x_train_low.shape)
print(x_test_low.shape)
print(y_train_low.shape)
print(y_test_low.shape)
print(x_train_high.shape)
print(x_test_high.shape)
print(y_train_high.shape)
print(y_test_high.shape)
print(len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high))

## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

gnb = GaussianNB()
gnb.fit(x_train_high, y_train_high)
y_pred_high = gnb.predict(x_test_high)

## #####################################################
## Model 2: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)

gb = GradientBoostingClassifier(random_state=42)
gb.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = gb.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt5 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt5[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(2677, 12)
(670, 12)
(2677,)
(670,)
(3515, 31)
(879, 31)
(3515,)
(879,)
7741


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.949287,0.868745,0.907232,0.841833,1379.0
1,All Obs,Depressed (Yes),0.369338,0.623529,0.463895,0.841833,170.0
2,All Obs,Macro Avg,0.659312,0.746137,0.685564,0.841833,1549.0
3,All Obs,Weighted Avg,0.885638,0.841833,0.858577,0.841833,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.967239,0.968750,0.967994,0.938806,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.310345,0.300000,0.305085,0.938806,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.638792,0.634375,0.636539,0.938806,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.937826,0.938806,0.938311,0.938806,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.930757,0.782138,0.850000,0.767918,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.375969,0.692857,0.487437,0.767918,140.0


# Opt 6: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GNB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2
- Model 2: 
 - GNB trained on observations with 0 dep screener response. 
 - Uses separate features. 
 - uses random under sampler

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [17]:
# model_features_low_opt6 = [
#     'respiratory_issues',
#     'seen_mental_health_professional',
#     'walk_or_bicycle',
#     'arthritis',
#     'has_smoked_tabacco_last_5days',
#     'work_schedule',
#     'currently_increase_exercise',
#     'over_35_hrs_worked',
#     'chest_discomfort'
# ]

# x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
#     original_df = cdc_survey_pmom,
#     columns_to_use_low = model_features_low_opt6,
#     columns_to_use_high = model_features_opt2
# )

# print(x_train_low.shape)
# print(x_test_low.shape)
# print(y_train_low.shape)
# print(y_test_low.shape)
# print(x_train_high.shape)
# print(x_test_high.shape)
# print(y_train_high.shape)
# print(y_test_high.shape)
# print(len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high))

# ## #####################################################
# ## Model 1: Based on observations with 1+ dep screener non-zero response

# gnb = GaussianNB()
# gnb.fit(x_train_high, y_train_high)
# y_pred_high = gnb.predict(x_test_high)

# ## #####################################################
# ## Model 2: Based on observations with 0 dep screener non-zero response

# # partially correct for class imbalance
# rus = RandomUnderSampler(
#     random_state=42, 
#     sampling_strategy=0.12,
#     replacement=False
# )
# x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)

# gnb_low = GaussianNB()
# gnb_low.fit(x_train_low_rus, y_train_low_rus)
# y_pred_low = gnb_low.predict(x_test_low)

# ## #####################################################
# ## Predict: 

# y_pred = np.concatenate((y_pred_high, y_pred_low))
# y_actual = np.concatenate((y_test_high, y_test_low))

# # Performance All
# ensemble_performance_all = get_performance_df(
#     label_actual = y_actual, 
#     label_pred = y_pred,
#     model_name = 'All Obs'
# )

# # Performance Model 1
# ensemble_performance_high = get_performance_df(
#     label_actual = y_test_high, 
#     label_pred = y_pred_high,
#     model_name = 'Obs w/ 1+ Dep Screener'
# )

# # Performance Model 2
# ensemble_performance_low = get_performance_df(
#     label_actual = y_test_low, 
#     label_pred = y_pred_low,
#     model_name = 'Obs w/ 0 Dep Screener'
# )

# ## #####################################################
# ## Performance
# ensemble_performance_opt6 = pd.concat([
#     ensemble_performance_all,
#     ensemble_performance_high,
#     ensemble_performance_low
# ])

# cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
# ensemble_performance_opt6[cols].sort_values(by=['model','grouping']) \
#     .rename(columns = {'model':'Observations (Obs)'})

# Opt 7: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GNB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2
- Model 2: 
 - RF trained on observations with 0 dep screener response. 
 - Uses separate features (one binned column)
 - uses random under sampler

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [18]:
# cdc_survey_pmom.groupby('count_days_seen_doctor_12mo_bin') \
#     .agg({'MDD':'mean','count_days_seen_doctor_12mo':'max','SEQN':'count'}) \
#     .reset_index()

In [19]:
## ##############################
## Create count_days_seen_doctor_12mo_bin

def create_bin_lookup(
        feature,
        n_bins,
        encode,
        strategy,
        df_to_use):
        # make a new column with _bin suffix
        new_column_name = feature + '_bin'

        # get non-null values per column
        feature_values = df_to_use[feature].dropna()

        # reshape to be 1 column
        feature_values = feature_values.to_numpy().reshape([feature_values.shape[0],1])

        # create bins using estimator
        est = KBinsDiscretizer(
            n_bins=n_bins,
            encode=encode, 
            strategy=strategy, 
            subsample=None
        )
        est.fit(feature_values)
        feature_values_bin = pd.DataFrame(est.transform(feature_values))

        # dataframe with binned values
        feature_values_bin.columns = [new_column_name]

        # get original
        feature_values = pd.DataFrame(feature_values)
        feature_values.columns = ['original']

        # merge bin & non-binned values together to make a lookup
        feature_values = feature_values.merge(feature_values_bin, left_index=True, right_index=True)
        feature_value_bin_lookup = feature_values.groupby(['original',new_column_name]).count().reset_index()

        return feature_value_bin_lookup, new_column_name
    
feature_value_bin_lookup, new_column_name = create_bin_lookup(
    'count_days_seen_doctor_12mo',
    n_bins=10,
    encode='ordinal',
    strategy='uniform',
    df_to_use=cdc_survey_pmom
)  

# prevent creating a column if already exists
# happens if you run this block multiple times
if new_column_name in cdc_survey_pmom.columns:
    cdc_survey_pmom.drop(columns=new_column_name,inplace=True)

# add bin column in a way that doesn't drop nulls
cdc_survey_pmom = cdc_survey_pmom.merge(
    feature_value_bin_lookup, 
    left_on='count_days_seen_doctor_12mo', 
    right_on='original', 
    how = 'left'
)

# drop column called "original" as was only used to join
cdc_survey_pmom.drop(columns=['original'], inplace=True)  


model_features_low_opt7 = [
    'count_days_seen_doctor_12mo_bin',
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'work_schedule'
]

x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_low_opt7,
    columns_to_use_high = model_features_opt2
)

print(f"x_train_low: {x_train_low.shape}")
print(f"y_train_low: {y_train_low.shape}")
print(f"x_train_high: {x_train_high.shape}")
print(f"y_train_high: {y_train_high.shape}")
print(f"x_test_low: {x_test_low.shape}")
print(f"y_test_low: {y_test_low.shape}")
print(f"x_test_high: {x_test_high.shape}")
print(f"y_test_high: {y_test_high.shape}")
print(f"total_obs: {len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high)}")


## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

gnb = GaussianNB()
gnb.fit(x_train_high, y_train_high)
y_pred_high = gnb.predict(x_test_high)

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)
print(f"x_train_low_rus: {x_train_low_rus.shape}")

rf = RandomForestClassifier(random_state=42)
rf.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = rf.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt7 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt7[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

x_train_low: (2677, 11)
y_train_low: (2677,)
x_train_high: (3515, 31)
y_train_high: (3515,)
x_test_low: (670, 11)
y_test_low: (670,)
x_test_high: (879, 31)
y_test_high: (879,)
total_obs: 7741
x_train_low_rus: (849, 11)


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.948859,0.874547,0.910189,0.846352,1379.0
1,All Obs,Depressed (Yes),0.377698,0.617647,0.468750,0.846352,170.0
2,All Obs,Macro Avg,0.663279,0.746097,0.689469,0.846352,1549.0
3,All Obs,Weighted Avg,0.886175,0.846352,0.861742,0.846352,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.966154,0.981250,0.973643,0.949254,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.400000,0.266667,0.320000,0.949254,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.683077,0.623958,0.646822,0.949254,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.940804,0.949254,0.944376,0.949254,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.930757,0.782138,0.850000,0.767918,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.375969,0.692857,0.487437,0.767918,140.0


### Error Analysis for model on obs with 1+ dep screener

In [20]:
def get_test_pred_data(model, 
                       idx_test_to_use,
                       x_test_to_use, 
                       y_test_to_use,
                       y_pred_to_use,
                       features_to_use):
    
    ## ##########################
    ## predicted prob along with model features

    pred_prob = pd.DataFrame(model.predict_proba(x_test_to_use))
    pred_prob.columns = ['depressed_no_prob','depressed_yes_prob']
    pred_prob['y_pred'] = y_pred_to_use
    pred_prob['y_actual'] = y_test_to_use

    pred_prob_with_feat = pd.concat([
        pred_prob, 
        # column bind test observations with their original values
        cdc_survey_pmom.iloc[idx_test_to_use][features_to_use].reset_index()
    ], axis=1)

    ## ##########################
    ## bin probability

    # bin the probabilities into groups of 10%
    bin_labels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    pred_prob_with_feat['depressed_yes_prob_bin'] = pd.cut(
        pred_prob_with_feat['depressed_yes_prob'], 
        bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, np.inf],
        labels=bin_labels
    )

    # bin the probabilities into custom groups
    bin_labels = ['[0,0.1)', '[0.1, 0.3)', '[0.3, 0.5)', '[0.5, 0.7)', '[0.7, 0.9)', '[0.9, 1]']
    pred_prob_with_feat['depressed_yes_prob_bin_v2'] = pd.cut(
        pred_prob_with_feat['depressed_yes_prob'], 
        bins=[0, 0.1, 0.3, 0.5, 0.7, 0.9, np.inf],
        labels=bin_labels
    )

    ## ##########################
    ## create additional columns

    def label_pred(row):
        """
        Function that labels prediction cases into TP, TN, FP, FN
        """
        if row['y_actual'] == row['y_pred']:
            if row['y_actual'] == 1:
                result = 'TP'
            else:
                result = 'TN'
        else: 
            if row['y_actual'] == 1:
                result = 'FN'
            else:
                result = 'FP'

        return result

    # calculate prediction label
    pred_prob_with_feat['label'] = pred_prob_with_feat.apply(label_pred, axis = 1)
    pred_prob_with_feat['count_FN'] = pred_prob_with_feat['label'] == "FN"
    pred_prob_with_feat['count_TN'] = pred_prob_with_feat['label'] == "TN"
    pred_prob_with_feat['count_FP'] = pred_prob_with_feat['label'] == "FP"
    pred_prob_with_feat['count_TP'] = pred_prob_with_feat['label'] == "TP"
    pred_prob_with_feat['sample_size'] = 1

    pred_prob_with_feat['num_dep_screener_0'] = (pred_prob_with_feat[dep_screener_cols]==0).sum(axis=1)
    pred_prob_with_feat['num_dep_screener_1'] = (pred_prob_with_feat[dep_screener_cols]==1).sum(axis=1)
    pred_prob_with_feat['num_dep_screener_2'] = (pred_prob_with_feat[dep_screener_cols]==2).sum(axis=1)
    pred_prob_with_feat['num_dep_screener_3'] = (pred_prob_with_feat[dep_screener_cols]==3).sum(axis=1)
    return pred_prob_with_feat

def summarize_predictions_by_col(
    dt_to_use, 
    group_by_col):
    # summarize observations by predicted probability
    cols = ['y_pred','y_actual','count_FN','count_TN','count_FP','count_TP','sample_size']
    dt_to_use = dt_to_use.groupby([group_by_col])[cols].sum().reset_index()
    dt_to_use['recall_PC'] = dt_to_use['count_TP'] / (dt_to_use['count_TP'] + dt_to_use['count_FN'])
    dt_to_use['recall_NC'] = dt_to_use['count_TN'] / (dt_to_use['count_TN'] + dt_to_use['count_FP'])
    dt_to_use['accuracy'] = (dt_to_use['count_TP'] + dt_to_use['count_TN']) / dt_to_use['sample_size']
    dt_to_use['precision_PC'] = dt_to_use['count_TP'] / (dt_to_use['count_TP'] + dt_to_use['count_FP'])
    dt_to_use['pct_mdd'] = dt_to_use['y_actual'] / dt_to_use['sample_size']
    dt_to_use['sample_size (% of total)'] = dt_to_use['sample_size'] / dt_to_use['sample_size'].sum()
    return dt_to_use

In [21]:
x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9],
    columns = model_features_opt2,
    return_indices = True
)

print(f"x_train: {x_train.shape}")
print(f"x_test: {x_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

gnb_opt7 = GaussianNB()
gnb_opt7.fit(x_train, y_train)
gnb_opt7_y_pred = gnb_opt7.predict(x_test)
gnb_opt7_score = get_performance_df(y_test, gnb_opt7_y_pred,'Gaussian Naive Bayes')  
gnb_opt7_score

x_train: (3515, 31)
x_test: (879, 31)
y_train: (3515,)
y_test: (879,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes,Depressed (No),0.930757,0.782138,0.850000,739.0,0.767918
1,Gaussian Naive Bayes,Depressed (Yes),0.375969,0.692857,0.487437,140.0,0.767918
2,Gaussian Naive Bayes,Macro Avg,0.653363,0.737498,0.668719,879.0,0.767918
3,Gaussian Naive Bayes,Weighted Avg,0.842395,0.767918,0.792254,879.0,0.767918


In [39]:
gnb_opt7_pred_data = get_test_pred_data(
    model=gnb_opt7, 
    idx_test_to_use=idx_test,
    x_test_to_use=x_test, 
    y_test_to_use=y_test,
    y_pred_to_use=gnb_opt7_y_pred,
    features_to_use=model_features_opt2
)

cols = ['num_dep_screener_0','num_dep_screener_1','num_dep_screener_2','num_dep_screener_3']
gnb_opt7_pred_data.groupby(['depressed_yes_prob_bin_v2'])[cols].mean().round(3).reset_index()

ValueError: X has 10 features, but GaussianNB is expecting 31 features as input.

In [23]:
summarize_predictions_by_col(
    dt_to_use = gnb_opt7_pred_data,
    group_by_col = 'depressed_yes_prob_bin_v2'
)

,depressed_yes_prob_bin_v2,y_pred,y_actual,count_FN,count_TN,count_FP,count_TP,sample_size,recall_PC,recall_NC,accuracy,precision_PC,pct_mdd,sample_size (% of total)
0,"[0,0.1)",0,27,27,475,0,0,502,0.0,1.0,0.946215,NaN,0.053785,0.571104
1,"[0.1, 0.3)",0,9,9,68,0,0,77,0.0,1.0,0.883117,NaN,0.116883,0.087600
2,"[0.3, 0.5)",0,7,7,35,0,0,42,0.0,1.0,0.833333,NaN,0.166667,0.047782
3,"[0.5, 0.7)",39,14,0,0,25,14,39,1.0,0.0,0.358974,0.358974,0.358974,0.044369
4,"[0.7, 0.9)",32,7,0,0,25,7,32,1.0,0.0,0.218750,0.218750,0.218750,0.036405
5,"[0.9, 1]",187,76,0,0,111,76,187,1.0,0.0,0.406417,0.406417,0.406417,0.212742


In [24]:
summarize_predictions_by_col(
    dt_to_use = gnb_opt7_pred_data,
    group_by_col = 'num_dep_screener_0'
)

,num_dep_screener_0,y_pred,y_actual,count_FN,count_TN,count_FP,count_TP,sample_size,recall_PC,recall_NC,accuracy,precision_PC,pct_mdd,sample_size (% of total)
0,0,4,1,0,9,3,1,13,1.000000,0.750000,0.769231,0.250000,0.076923,0.014790
1,1,8,1,0,14,7,1,22,1.000000,0.666667,0.681818,0.125000,0.045455,0.025028
2,2,9,5,1,17,5,4,27,0.800000,0.772727,0.777778,0.444444,0.185185,0.030717
3,3,17,12,1,25,6,11,43,0.916667,0.806452,0.837209,0.647059,0.279070,0.048919
4,4,16,6,1,25,11,5,42,0.833333,0.694444,0.714286,0.312500,0.142857,0.047782
5,5,16,8,2,29,10,6,47,0.750000,0.743590,0.744681,0.375000,0.170213,0.053470
6,6,22,11,5,43,16,6,70,0.545455,0.728814,0.700000,0.272727,0.157143,0.079636
7,7,23,18,8,66,13,10,97,0.555556,0.835443,0.783505,0.434783,0.185567,0.110353
8,8,32,17,6,87,21,11,125,0.647059,0.805556,0.784000,0.343750,0.136000,0.142207
9,9,111,61,19,263,69,42,393,0.688525,0.792169,0.776081,0.378378,0.155216,0.447099


# Opt 8: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GNB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2. 
 - Uses oversampler.
- Model 2: 
 - RF trained on observations with 0 dep screener response. 
 - Uses features from opt 7. 
 - Uses undersampler

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [25]:
x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_low_opt7,
    columns_to_use_high = model_features_opt2
)

print(f"x_train_low: {x_train_low.shape}")
print(f"y_train_low: {y_train_low.shape}")
print(f"x_train_high: {x_train_high.shape}")
print(f"y_train_high: {y_train_high.shape}")
print(f"x_test_low: {x_test_low.shape}")
print(f"y_test_low: {y_test_low.shape}")
print(f"x_test_high: {x_test_high.shape}")
print(f"y_test_high: {y_test_high.shape}")
print(f"total_obs: {len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high)}")


## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

# partially correct for class imbalance
ros = RandomOverSampler(
    random_state=42, 
    sampling_strategy=0.33
)
x_train_high_ros, y_train_high_ros = ros.fit_resample(x_train_high,y_train_high)

gnb = GaussianNB()
gnb.fit(x_train_high_ros, y_train_high_ros)
y_pred_high = gnb.predict(x_test_high)
print(f"x_train_high_ros: {x_train_high_ros.shape}")

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)
print(f"x_train_low_rus: {x_train_low_rus.shape}")

rf = RandomForestClassifier(random_state=42)
rf.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = rf.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt8 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt8[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

x_train_low: (2677, 11)
y_train_low: (2677,)
x_train_high: (3515, 31)
y_train_high: (3515,)
x_test_low: (670, 11)
y_test_low: (670,)
x_test_high: (879, 31)
y_test_high: (879,)
total_obs: 7741
x_train_high_ros: (4021, 31)
x_train_low_rus: (849, 11)


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.957535,0.833938,0.891473,0.819238,1379.0
1,All Obs,Depressed (Yes),0.341954,0.700000,0.459459,0.819238,170.0
2,All Obs,Macro Avg,0.649745,0.766969,0.675466,0.819238,1549.0
3,All Obs,Weighted Avg,0.889976,0.819238,0.844060,0.819238,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.966154,0.981250,0.973643,0.949254,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.400000,0.266667,0.320000,0.949254,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.683077,0.623958,0.646822,0.949254,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.940804,0.949254,0.944376,0.949254,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.947368,0.706360,0.809302,0.720137,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.338415,0.792857,0.474359,0.720137,140.0


### Error Analysis for model on obs with 1+ dep screener

In [26]:
x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9],
    columns = model_features_opt2,
    return_indices = True
)

print(f"x_train: {x_train.shape}")
print(f"x_test: {x_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

ros = RandomOverSampler(
    random_state=42, 
    sampling_strategy=0.33
)
x_train_high_ros, y_train_high_ros = ros.fit_resample(x_train_high,y_train_high)
print(f"x_train_high_ros: {x_train_high_ros.shape}")


gnb_opt8 = GaussianNB()
gnb_opt8.fit(x_train_high_ros, y_train_high_ros)
gnb_opt8_y_pred = gnb_opt8.predict(x_test)
gnb_opt8_score = get_performance_df(y_test, gnb_opt8_y_pred,'Gaussian Naive Bayes')  
gnb_opt8_score

x_train: (3515, 31)
x_test: (879, 31)
y_train: (3515,)
y_test: (879,)
x_train_high_ros: (4021, 31)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes,Depressed (No),0.947368,0.706360,0.809302,739.0,0.720137
1,Gaussian Naive Bayes,Depressed (Yes),0.338415,0.792857,0.474359,140.0,0.720137
2,Gaussian Naive Bayes,Macro Avg,0.642892,0.749609,0.641831,879.0,0.720137
3,Gaussian Naive Bayes,Weighted Avg,0.850379,0.720137,0.755955,879.0,0.720137


In [27]:
gnb_opt8_pred_data = get_test_pred_data(
    model=gnb_opt8, 
    idx_test_to_use=idx_test,
    x_test_to_use=x_test, 
    y_test_to_use=y_test,
    y_pred_to_use=gnb_opt8_y_pred,
    features_to_use=model_features_opt2
)

cols = ['num_dep_screener_0','num_dep_screener_1','num_dep_screener_2','num_dep_screener_3']
gnb_opt8_pred_data.groupby(['depressed_yes_prob_bin_v2'])[cols].median().round(3).reset_index()

,depressed_yes_prob_bin_v2,num_dep_screener_0,num_dep_screener_1,num_dep_screener_2,num_dep_screener_3
0,"[0,0.1)",8.0,1.0,0.0,0.0
1,"[0.1, 0.3)",8.0,1.0,0.0,0.0
2,"[0.3, 0.5)",8.0,1.0,0.0,0.0
3,"[0.5, 0.7)",8.0,2.0,0.0,0.0
4,"[0.7, 0.9)",9.0,1.0,0.0,0.0
5,"[0.9, 1]",8.0,1.0,0.0,0.0


In [28]:
summarize_predictions_by_col(
    dt_to_use = gnb_opt8_pred_data,
    group_by_col = 'depressed_yes_prob_bin_v2'
)

,depressed_yes_prob_bin_v2,y_pred,y_actual,count_FN,count_TN,count_FP,count_TP,sample_size,recall_PC,recall_NC,accuracy,precision_PC,pct_mdd,sample_size (% of total)
0,"[0,0.1)",0,18,18,399,0,0,417,0.0,1.0,0.956835,NaN,0.043165,0.474403
1,"[0.1, 0.3)",0,10,10,77,0,0,87,0.0,1.0,0.885057,NaN,0.114943,0.098976
2,"[0.3, 0.5)",0,1,1,46,0,0,47,0.0,1.0,0.978723,NaN,0.021277,0.053470
3,"[0.5, 0.7)",52,12,0,0,40,12,52,1.0,0.0,0.230769,0.230769,0.230769,0.059158
4,"[0.7, 0.9)",62,19,0,0,43,19,62,1.0,0.0,0.306452,0.306452,0.306452,0.070535
5,"[0.9, 1]",214,80,0,0,134,80,214,1.0,0.0,0.373832,0.373832,0.373832,0.243458


In [29]:
summarize_predictions_by_col(
    dt_to_use = gnb_opt8_pred_data,
    group_by_col = 'num_dep_screener_0'
)

,num_dep_screener_0,y_pred,y_actual,count_FN,count_TN,count_FP,count_TP,sample_size,recall_PC,recall_NC,accuracy,precision_PC,pct_mdd,sample_size (% of total)
0,0,4,1,0,9,3,1,13,1.000000,0.750000,0.769231,0.250000,0.076923,0.014790
1,1,11,1,0,11,10,1,22,1.000000,0.523810,0.545455,0.090909,0.045455,0.025028
2,2,10,5,0,17,5,5,27,1.000000,0.772727,0.814815,0.500000,0.185185,0.030717
3,3,19,12,0,24,7,12,43,1.000000,0.774194,0.837209,0.631579,0.279070,0.048919
4,4,21,6,1,20,16,5,42,0.833333,0.555556,0.595238,0.238095,0.142857,0.047782
5,5,19,8,1,27,12,7,47,0.875000,0.692308,0.723404,0.368421,0.170213,0.053470
6,6,27,11,5,38,21,6,70,0.545455,0.644068,0.628571,0.222222,0.157143,0.079636
7,7,32,18,3,62,17,15,97,0.833333,0.784810,0.793814,0.468750,0.185567,0.110353
8,8,42,17,4,79,29,13,125,0.764706,0.731481,0.736000,0.309524,0.136000,0.142207
9,9,143,61,15,235,97,46,393,0.754098,0.707831,0.715013,0.321678,0.155216,0.447099


# Opt 9: Ensemble Model

Build 2 models with different feature set
- Model 1: 
 - GB trained on observations with 1+ dep screener response. 
 - Uses features from opt 2. 
 - Uses undersampler.
- Model 2: 
 - RF trained on observations with 0 dep screener response. 
 - Uses features from opt 7. 
 - Uses undersampler

Notes
- _low = has 9+ dep screeners answered 0
- _high = has <9 dep screeners answered 0

In [30]:
x_train_low, x_test_low, y_train_low, y_test_low, x_train_high, x_test_high, y_train_high, y_test_high = get_model_data_ensemble_v2(
    original_df = cdc_survey_pmom,
    columns_to_use_low = model_features_low_opt7,
    columns_to_use_high = model_features_opt2
)

print(f"x_train_low: {x_train_low.shape}")
print(f"y_train_low: {y_train_low.shape}")
print(f"x_train_high: {x_train_high.shape}")
print(f"y_train_high: {y_train_high.shape}")
print(f"x_test_low: {x_test_low.shape}")
print(f"y_test_low: {y_test_low.shape}")
print(f"x_test_high: {x_test_high.shape}")
print(f"y_test_high: {y_test_high.shape}")
print(f"total_obs: {len(y_test_low) + len(y_test_high) + len(y_train_low) + len(y_train_high)}")

## #####################################################
## Model 1: Based on observations with 1+ dep screener non-zero response

# partially correct for class imbalance
rus_model1 = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=1,
    replacement=False
)
x_train_high_rus, y_train_high_rus = rus_model1.fit_resample(x_train_high,y_train_high)
print(f"x_train_high_rus: {x_train_high_rus.shape}")

gb = GradientBoostingClassifier(random_state=42)
gb.fit(x_train_high_rus, y_train_high_rus)
y_pred_high = gb.predict(x_test_high)

## #####################################################
## Model 1: Based on observations with 0 dep screener non-zero response

# partially correct for class imbalance
rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)
x_train_low_rus, y_train_low_rus = rus.fit_resample(x_train_low,y_train_low)
print(f"x_train_low_rus: {x_train_low_rus.shape}")

rf = RandomForestClassifier(random_state=42)
rf.fit(x_train_low_rus, y_train_low_rus)
y_pred_low = rf.predict(x_test_low)

## #####################################################
## Predict: 

y_pred = np.concatenate((y_pred_high, y_pred_low))
y_actual = np.concatenate((y_test_high, y_test_low))

# Performance All
ensemble_performance_all = get_performance_df(
    label_actual = y_actual, 
    label_pred = y_pred,
    model_name = 'All Obs'
)

# Performance Model 1
ensemble_performance_high = get_performance_df(
    label_actual = y_test_high, 
    label_pred = y_pred_high,
    model_name = 'Obs w/ 1+ Dep Screener'
)

# Performance Model 2
ensemble_performance_low = get_performance_df(
    label_actual = y_test_low, 
    label_pred = y_pred_low,
    model_name = 'Obs w/ 0 Dep Screener'
)

## #####################################################
## Performance
ensemble_performance_opt9 = pd.concat([
    ensemble_performance_all,
    ensemble_performance_high,
    ensemble_performance_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
ensemble_performance_opt9[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

x_train_low: (2677, 11)
y_train_low: (2677,)
x_train_high: (3515, 31)
y_train_high: (3515,)
x_test_low: (670, 11)
y_test_low: (670,)
x_test_high: (879, 31)
y_test_high: (879,)
total_obs: 7741
x_train_high_rus: (982, 31)
x_train_low_rus: (849, 11)


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.956664,0.848441,0.899308,0.830859,1379.0
1,All Obs,Depressed (Yes),0.358896,0.688235,0.471774,0.830859,170.0
2,All Obs,Macro Avg,0.657780,0.768338,0.685541,0.830859,1549.0
3,All Obs,Weighted Avg,0.891060,0.830859,0.852387,0.830859,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.966154,0.981250,0.973643,0.949254,640.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.400000,0.266667,0.320000,0.949254,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.683077,0.623958,0.646822,0.949254,670.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.940804,0.949254,0.944376,0.949254,670.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.945899,0.733424,0.826220,0.740614,739.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.356209,0.778571,0.488789,0.740614,140.0


### Error Analysis for model on obs with 1+ dep screener

In [31]:
x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9],
    columns = model_features_opt2,
    return_indices = True
)

rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=1,
    replacement=False
)
x_train_rus, y_train_rus = rus.fit_resample(x_train,y_train)

print(f"x_train: {x_train.shape}")
print(f"x_test: {x_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"x_train_rus: {x_train_rus.shape}")

gb_opt9 = GradientBoostingClassifier(random_state=42)
gb_opt9.fit(x_train_rus, y_train_rus)
#gb_opt9.fit(x_train, y_train)
gb_opt9_y_pred  = gb_opt9.predict(x_test)
gb_opt9_score = get_performance_df(y_test, gb_opt9_y_pred,'Gradient Boosting Classifier')  
gb_opt9_score

x_train: (3515, 31)
x_test: (879, 31)
y_train: (3515,)
y_test: (879,)
x_train_rus: (982, 31)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gradient Boosting Classifier,Depressed (No),0.945899,0.733424,0.826220,739.0,0.740614
1,Gradient Boosting Classifier,Depressed (Yes),0.356209,0.778571,0.488789,140.0,0.740614
2,Gradient Boosting Classifier,Macro Avg,0.651054,0.755997,0.657504,879.0,0.740614
3,Gradient Boosting Classifier,Weighted Avg,0.851978,0.740614,0.772476,879.0,0.740614


In [32]:
pd.DataFrame({'feature': model_features_opt2,'coefficient': gb_opt9.feature_importances_}) \
    .sort_values(by='coefficient',ascending=False)

,feature,coefficient
10,seen_mental_health_professional,0.175818
29,num_dep_screener_0,0.144224
1,feeling_down_depressed_hopeless,0.117758
15,age_in_years,0.100797
30,weight_lbs_over_height_in_ratio,0.086126
25,is_usa_born,0.036154
13,cholesterol_prescription,0.036066
16,horomones_not_bc,0.033994
9,difficult_doing_daytoday_tasks,0.032573
22,thyroid_issues,0.028418


In [33]:
gb_opt9_pred_data = get_test_pred_data(
    model=gb_opt9, 
    idx_test_to_use=idx_test,
    x_test_to_use=x_test, 
    y_test_to_use=y_test,
    y_pred_to_use=gb_opt9_y_pred,
    features_to_use=model_features_opt2
)

cols = ['num_dep_screener_0','num_dep_screener_1','num_dep_screener_2','num_dep_screener_3']
gb_opt9_pred_data.groupby(['depressed_yes_prob_bin_v2'])[cols].median().round(3).reset_index()

,depressed_yes_prob_bin_v2,num_dep_screener_0,num_dep_screener_1,num_dep_screener_2,num_dep_screener_3
0,"[0,0.1)",8.0,1.0,0.0,0.0
1,"[0.1, 0.3)",8.0,1.0,0.0,0.0
2,"[0.3, 0.5)",8.0,1.0,0.0,0.0
3,"[0.5, 0.7)",8.0,1.0,0.0,0.0
4,"[0.7, 0.9)",8.0,1.0,0.0,0.0
5,"[0.9, 1]",8.0,1.0,0.0,0.0


In [34]:
summarize_predictions_by_col(
    dt_to_use = gb_opt9_pred_data,
    group_by_col = 'depressed_yes_prob_bin_v2'
)

,depressed_yes_prob_bin_v2,y_pred,y_actual,count_FN,count_TN,count_FP,count_TP,sample_size,recall_PC,recall_NC,accuracy,precision_PC,pct_mdd,sample_size (% of total)
0,"[0,0.1)",0,2,2,97,0,0,99,0.0,1.0,0.979798,NaN,0.020202,0.112628
1,"[0.1, 0.3)",0,12,12,261,0,0,273,0.0,1.0,0.956044,NaN,0.043956,0.310580
2,"[0.3, 0.5)",0,17,17,184,0,0,201,0.0,1.0,0.915423,NaN,0.084577,0.228669
3,"[0.5, 0.7)",147,42,0,0,105,42,147,1.0,0.0,0.285714,0.285714,0.285714,0.167235
4,"[0.7, 0.9)",124,48,0,0,76,48,124,1.0,0.0,0.387097,0.387097,0.387097,0.141069
5,"[0.9, 1]",35,19,0,0,16,19,35,1.0,0.0,0.542857,0.542857,0.542857,0.039818


In [35]:
summarize_predictions_by_col(
    dt_to_use = gb_opt9_pred_data,
    group_by_col = 'num_dep_screener_0'
)

,num_dep_screener_0,y_pred,y_actual,count_FN,count_TN,count_FP,count_TP,sample_size,recall_PC,recall_NC,accuracy,precision_PC,pct_mdd,sample_size (% of total)
0,0,4,1,0,9,3,1,13,1.000000,0.750000,0.769231,0.250000,0.076923,0.014790
1,1,7,1,0,15,6,1,22,1.000000,0.714286,0.727273,0.142857,0.045455,0.025028
2,2,11,5,0,16,6,5,27,1.000000,0.727273,0.777778,0.454545,0.185185,0.030717
3,3,17,12,2,24,7,10,43,0.833333,0.774194,0.790698,0.588235,0.279070,0.048919
4,4,16,6,1,25,11,5,42,0.833333,0.694444,0.714286,0.312500,0.142857,0.047782
5,5,18,8,1,28,11,7,47,0.875000,0.717949,0.744681,0.388889,0.170213,0.053470
6,6,26,11,3,41,18,8,70,0.727273,0.694915,0.700000,0.307692,0.157143,0.079636
7,7,29,18,7,61,18,11,97,0.611111,0.772152,0.742268,0.379310,0.185567,0.110353
8,8,40,17,3,82,26,14,125,0.823529,0.759259,0.768000,0.350000,0.136000,0.142207
9,9,138,61,14,241,91,47,393,0.770492,0.725904,0.732824,0.340580,0.155216,0.447099


# Baseline For Comparison

In [36]:
## #####################################################
## Build Model

x_train, x_test, y_train, y_test, idx_train, idx_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols,
    return_indices = True
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

bnb_base = BernoulliNB()
bnb_base.fit(x_train, y_train)
bnb_base_pred  = bnb_base.predict(x_test)

## #####################################################
## Get Features to identify performance across low & high dep screener response

bnb_base_pred_prob = pd.DataFrame(bnb_base.predict_proba(x_test))
bnb_base_pred_prob.columns = ['depressed_no_prob','depressed_yes_prob']
bnb_base_pred_prob['y_pred'] = bnb_base_pred
bnb_base_pred_prob['y_actual'] = y_test

bnb_base_pred_prob_with_feat = pd.concat([
    bnb_base_pred_prob, 
    # column bind test observations with their original values
    cdc_survey_pmom.iloc[idx_test][dep_screener_cols].reset_index()
], axis=1)

bnb_base_pred_prob_with_feat['num_dep_screener_0'] = (bnb_base_pred_prob_with_feat[dep_screener_cols]==0).sum(axis=1)

# 0 dep screener response
bnb_base_pred_low = bnb_base_pred_prob_with_feat[
    bnb_base_pred_prob_with_feat['num_dep_screener_0'] >= 9
]

# 1+ dep screener response
bnb_base_pred_high = bnb_base_pred_prob_with_feat[
    bnb_base_pred_prob_with_feat['num_dep_screener_0'] < 9
]

## #####################################################
## Cal scores

bnb_base_score_all = get_performance_df(y_test, bnb_base_pred,'All Obs')  

bnb_base_score_high = get_performance_df(
    label_actual = bnb_base_pred_high['y_actual'], 
    label_pred = bnb_base_pred_high['y_pred'],
    model_name = 'Obs w/ 1+ Dep Screener'
)

bnb_base_score_low = get_performance_df(
    label_actual = bnb_base_pred_low['y_actual'], 
    label_pred = bnb_base_pred_low['y_pred'],
    model_name = 'Obs w/ 0 Dep Screener'
)

baseline_performance = pd.concat([
    bnb_base_score_all,
    bnb_base_score_high,
    bnb_base_score_low
])

cols = ['model','grouping','precision','recall','f1-score','accuracy','support']
baseline_performance[cols].sort_values(by=['model','grouping']) \
    .rename(columns = {'model':'Observations (Obs)'})

(6192, 10)
(1549, 10)
(6192,)
(1549,)


,Observations (Obs),grouping,precision,recall,f1-score,accuracy,support
0,All Obs,Depressed (No),0.936236,0.866187,0.899851,0.826985,1390.0
1,All Obs,Depressed (Yes),0.292776,0.484277,0.364929,0.826985,159.0
2,All Obs,Macro Avg,0.614506,0.675232,0.632390,0.826985,1549.0
3,All Obs,Weighted Avg,0.870187,0.826985,0.844942,0.826985,1549.0
0,Obs w/ 0 Dep Screener,Depressed (No),0.956076,1.000000,0.977545,0.956076,653.0
1,Obs w/ 0 Dep Screener,Depressed (Yes),0.000000,0.000000,0.000000,0.956076,30.0
2,Obs w/ 0 Dep Screener,Macro Avg,0.478038,0.500000,0.488772,0.956076,683.0
3,Obs w/ 0 Dep Screener,Weighted Avg,0.914082,0.956076,0.934607,0.956076,683.0
0,Obs w/ 1+ Dep Screener,Depressed (No),0.913765,0.747626,0.822388,0.725173,737.0
1,Obs w/ 1+ Dep Screener,Depressed (Yes),0.292776,0.596899,0.392857,0.725173,129.0


# Data Dump For Comparison (Baseline vs Opt 5)

In [37]:
best_performance_melt = ensemble_performance_opt5.melt(id_vars=['model','grouping'])
best_performance_melt['context'] = 'best'
baseline_performance_melt = baseline_performance.melt(id_vars=['model','grouping'])
baseline_performance_melt['context'] = 'baseline'
performance_all = pd.concat([best_performance_melt,baseline_performance_melt])
performance_all.to_csv("best_vs_baseline_performance_base_vs_opt5.csv")

# Data Dump For Comparison (Baseline vs Opt 7)

In [38]:
best_performance_melt = ensemble_performance_opt7.melt(id_vars=['model','grouping'])
best_performance_melt['context'] = 'best'
baseline_performance_melt = baseline_performance.melt(id_vars=['model','grouping'])
baseline_performance_melt['context'] = 'baseline'
performance_all = pd.concat([best_performance_melt,baseline_performance_melt])
performance_all.to_csv("best_vs_baseline_performance_base_vs_opt7.csv")